In [1]:
import re
import ssl

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('patches_normalized.csv')
df

,version,champion,note,win_rate,impact,change,keyword,normalized_change
0,V10_1,Aphelios,Movement speed reduced to 325 from 330.,54.0,-1,5.0,movement speed,-0.177523
1,V10_1,Kassadin,Base movement speed reduced to 335 from 340.,50.0,-1,5.0,movement speed,-0.177523
2,V10_1,Mordekaiser,Bonus movement speed reduced to 3% at all leve...,49.0,-1,0.0,movement speed,-0.192595
3,V10_1,Sett,Active: Sett gains 30% bonus movement speed wi...,49.0,0,28.5,movement speed,-0.106684
4,V10_10,Miss Fortune,Maximum bonus movement speed reduced to 50 / 6...,46.0,-1,10.0,movement speed,-0.162451
...,...,...,...,...,...,...,...,...
14019,V13_6,Milio,NaN,NaN,0,NaN,Unknown,NaN
14020,V14_7,Skarner,NaN,NaN,0,NaN,Unknown,NaN
14021,V6_14,Yasuo,Bug Fix: Fixed a bug where Steel Tempest wasn'...,0.0,0,NaN,critical chance,NaN
14022,V6_2,Jhin,"Instead, Jhin gains attack damage from any att...",25.0,0,NaN,critical chance,NaN


In [6]:
versions = df['version'].unique()
versions

array(['V10_1', 'V10_10', 'V10_11', 'V10_12', 'V10_14', 'V10_15',
       'V10_16', 'V10_18', 'V10_19', 'V10_2', 'V10_22', 'V10_25', 'V10_3',
       'V10_4', 'V10_5', 'V10_6', 'V10_7', 'V10_8', 'V11_10', 'V11_11',
       'V11_13', 'V11_14', 'V11_15', 'V11_16', 'V11_17', 'V11_18',
       'V11_2', 'V11_20', 'V11_21', 'V11_3', 'V11_4', 'V11_5', 'V11_7',
       'V12_11', 'V12_12', 'V12_13', 'V12_15', 'V12_16', 'V12_18',
       'V12_2', 'V12_3', 'V12_4', 'V12_5', 'V12_8', 'V12_9', 'V13_1',
       'V13_10', 'V13_11', 'V13_12', 'V13_13', 'V13_14', 'V13_15',
       'V13_19', 'V13_24', 'V13_3', 'V13_4', 'V13_5', 'V13_6', 'V13_8',
       'V14_1', 'V14_2', 'V14_3', 'V14_4', 'V14_5', 'V14_6', 'V14_7',
       'V14_8', 'V5_23', 'V5_24', 'V6_10', 'V6_11', 'V6_13', 'V6_14',
       'V6_15', 'V6_16', 'V6_2', 'V6_23', 'V6_4', 'V6_5', 'V6_8', 'V7_1',
       'V7_10', 'V7_11', 'V7_12', 'V7_13', 'V7_14', 'V7_15', 'V7_16',
       'V7_17', 'V7_18', 'V7_2', 'V7_23', 'V7_3', 'V7_4', 'V7_6', 'V7_7',
       'V7_8',

In [18]:
versions = sorted(versions, key=lambda x: tuple([int(val) for val in x.replace('V', '').split('_')]))
versions, len(versions)

(['V5_23',
  'V5_24',
  'V6_1',
  'V6_2',
  'V6_3',
  'V6_4',
  'V6_5',
  'V6_7',
  'V6_8',
  'V6_10',
  'V6_11',
  'V6_12',
  'V6_13',
  'V6_14',
  'V6_15',
  'V6_16',
  'V6_18',
  'V6_21',
  'V6_23',
  'V7_1',
  'V7_2',
  'V7_3',
  'V7_4',
  'V7_5',
  'V7_6',
  'V7_7',
  'V7_8',
  'V7_10',
  'V7_11',
  'V7_12',
  'V7_13',
  'V7_14',
  'V7_15',
  'V7_16',
  'V7_17',
  'V7_18',
  'V7_22',
  'V7_23',
  'V8_1',
  'V8_2',
  'V8_3',
  'V8_4',
  'V8_5',
  'V8_6',
  'V8_8',
  'V8_10',
  'V8_11',
  'V8_12',
  'V8_13',
  'V8_14',
  'V8_15',
  'V8_16',
  'V8_17',
  'V8_19',
  'V8_20',
  'V8_21',
  'V8_22',
  'V8_23',
  'V8_24',
  'V9_1',
  'V9_2',
  'V9_3',
  'V9_4',
  'V9_5',
  'V9_6',
  'V9_7',
  'V9_8',
  'V9_10',
  'V9_11',
  'V9_12',
  'V9_13',
  'V9_14',
  'V9_15',
  'V9_17',
  'V9_19',
  'V9_21',
  'V9_23',
  'V9_24',
  'V10_1',
  'V10_2',
  'V10_3',
  'V10_4',
  'V10_5',
  'V10_6',
  'V10_7',
  'V10_8',
  'V10_10',
  'V10_11',
  'V10_12',
  'V10_13',
  'V10_14',
  'V10_15',
  'V10_16',


In [12]:
df['improved'] = np.nan
df.loc[df['version'] == 'V5_23'] = 0

In [36]:
for index, row in df.iterrows():
    if row['version'] == 'V5_23':
        df.at[index, 'improved'] = 0
        continue
    try:
        champion_name = row['champion']
        previous_version = versions[versions.index(row['version'])-1]
        previous_row = df.loc[(df['version']==previous_version) & (df['champion']==champion_name)]
        
        if len(previous_row) == 0:
            df.at[index, 'improved'] = 0
        else:
            previous_row = previous_row.iloc[0]

        if len(previous_row) > 0 and float(row['win_rate']) > float(previous_row['win_rate']):
            df.at[index, 'improved'] = 1
        else:
            df.at[index, 'improved'] = 0
    except Exception as e:
        print('error', e)

error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is not in list
error 0 is 

In [38]:
df[df['improved'] == 1.0]

,version,champion,note,win_rate,impact,change,keyword,normalized_change,improved
18,V10_16,Rakan,Initial bonus movement speed increased to 75% ...,62.0,1,25.0,movement speed,-0.117235,1.0
19,V10_16,Rakan,Undocumented: Bonus movement speed from collid...,62.0,-1,25.0,movement speed,-0.117235,1.0
96,V12_11,Zeri,Bonus movement speed per stack reduced to 1% f...,58.0,-1,0.5,movement speed,-0.191088,1.0
120,V12_3,Akshan,Base bonus movement speed reduced to 20 75 (b...,50.0,-1,55.0,movement speed,-0.026803,1.0
124,V12_3,Zeri,Base movement speed reduced to 325 from 330.,64.0,-1,5.0,movement speed,-0.177523,1.0
...,...,...,...,...,...,...,...,...,...
13890,V8_12,Kindred,Attack speed growth increased to 3.5% from 2.5%.,25.0,1,1.0,attack speed growth,-0.057835,1.0
13905,V9_6,Kayle,Attack speed growth reduced to 1.5% from 2%.,61.0,-1,0.5,attack speed growth,-0.203277,1.0
13906,V9_6,Neeko,Attack speed growth increased to 2.5% from 1.5%.,50.0,1,1.0,attack speed growth,-0.057835,1.0
13973,V7_6,Miss Fortune,New Effect: Cast range now adjusts based on Mi...,67.0,0,NaN,attack range,NaN,1.0
